# Problema de Otimização Não-Linear
## Projeto da disciplina **SME5720 - Otimização Não-linear**
### Estudo de caso da Regressão Linear

## Membros

* Eduardo Zaffari Monteiro - eduardozaffarimonteiro@usp.br - 12559490

* Gustavo Silva de Oliveira - gustavo.oliveira03@usp.br - 12567231

* Lucas Ivars Cadima Ciziks - luciziks@usp.br - 125599472

* Pedro Henrique de Freitas Maçonetto - pedromaconetto@usp.br - 12675419

## Introdução

Explicar qual o problema escolhido e os métodos que foram implementados, bem como que tipo de problemas eles resolvem.

## Modelagem do Problema

Descrever a modelagem matemática do problema e deverá ser apontado que tipo de método pode ser usado para resolvê-lo.

## Implementação

## Resultados Númericos

## Conclusão

## Referências

* https://probability4datascience.com/

* http://www.databookuw.com/

* Applied Linear Statistical Models

* INTRODUCTION TO LINEAR REGRESSION ANALYSIS